## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.


In [1]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import os

import pinon as pn
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

config = pn.Config('master')

Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data


In [2]:
dp = pn.DailyPrices()

Dataset "us-shareprices-daily" on disk (9 days old).
- Loading from disk ... 

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!


In [5]:
down = dp.downsample_prices('INTC')

In [6]:
down

,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
Date,,,,,,,,,
2000-03-31,85652.0,54.812381,53.767460,56.295714,55.077937,32.060317,0.01,5.561261e+07,NaN
2000-06-30,85652.0,62.524603,61.128571,64.117460,62.569841,36.429365,0.01,5.090721e+07,NaN
2000-09-30,85652.0,65.697619,64.301746,66.987460,65.426825,38.104286,0.02,4.714292e+07,NaN
2000-12-31,85652.0,39.474444,38.025556,40.578730,38.953333,22.694286,0.02,5.622823e+07,NaN
2001-03-31,85652.0,31.775968,30.769839,32.816613,31.708226,18.483548,0.02,5.713449e+07,NaN
...,...,...,...,...,...,...,...,...,...
2022-03-31,85652.0,49.519839,48.785484,50.213710,49.564516,47.298065,0.36,4.021207e+07,4.067000e+09
2022-06-30,85652.0,43.398065,42.730806,43.900645,43.264677,41.612581,0.36,3.541272e+07,4.068968e+09
2022-09-30,85652.0,34.189062,33.652969,34.556094,34.094531,33.101250,0.36,4.032034e+07,4.099516e+09


In [ ]:
dps = dp.get_daily_prices(['XOM', 'CVX', 'MSFT'])

In [ ]:
msft_qtrly_prices = dp.downsample_prices('MSFT')
msft_qtrly_prices_ndx = msft_qtrly_prices.index
msft_qtrly_prices

In [ ]:
qf = pn.Fundamentals()

In [ ]:
temp = qf.get_income_statements(['IBM','CVX', 'D', 'V', 'MA', 'MSFT'])
msft_stmnts = temp.loc['MSFT']
msft_stmnts_ndx = msft_stmnts.index
msft_stmnts

In [ ]:
temp.index

In [ ]:
re_prices = msft_qtrly_prices.reindex(msft_stmnts_ndx)
re_prices

In [ ]:
common_ndx = msft_stmnts_ndx.intersection(msft_qtrly_prices_ndx)
common_ndx

In [ ]:
re_statements = msft_stmnts.reindex(common_ndx)
re_prices = msft_qtrly_prices.reindex(common_ndx)

In [ ]:
msft_res = pd.DataFrame(index=common_ndx)
msft_res['eps_qtr'] = msft_stmnts[sf_cols.NET_INCOME] / msft_stmnts[sf_cols.SHARES_DILUTED]
msft_res['eps_ttm'] = msft_res['eps_qtr'].rolling(4).sum()
msft_res['pe'] = msft_qtrly_prices[sf_cols.CLOSE] / msft_res['eps_ttm']
msft_res